# GPT from scratch

このファイルではGPTモデルの実装を目標とする。このファイルの立ち位置は同リポジトリに含まれている<br>
・pytorch_command.ipynb<br>
・attention_from_scratch.ipynb<br>
の次に読むことを想定されている。pytorchの下位APIは上２つの.ipynbで散々練習したので、上位APIを解禁して最新モデルを組むことを目指す<br>

pytorchに明るい人はこのipynbファイルから読んでも良い、pytorchに明るくない人は上２つのipynbファイルを公式ドキュメントとにらめっこして読むことをおすすめする。<br>
内容がわかりにくいと感じた人はGithub上もしくはXで作者に質問/書き換え要請を投げることができる。<br>

##### 本題
実装においては[GPT from scratch](https://jaketae.github.io/study/gpt/)やKarpathyの[minGPT](https://github.com/karpathy/minGPT), [nanoGPT](https://github.com/karpathy/nanoGPT)を参考にする。<br>
実装に関してはこのipynbを見なくとも紹介した2つのサイトを見れば良い。<br>

実装に入る前に簡単なGPTの実装方針についての概要について解説する。今回実装するのは初代GPTである。<br>
初代GPTは以下のような構造をしている<br>
- GPTはTransformerのDecoderのみを用いたモデルである。
- Embedding_dimは768, MultiheadAttentionのHead数は12, TransformerDecoderブロック数は12である。
- FFN層の活性化関数はGELUである。

訓練時の注意事項も記載しておく
- OptimizerはAdam, 学習率の最大値は2.5e-4, 2000iteratonで最大値に達し、<br>その後はCOSINEスケジューラーによってスケジューリングを行う。attention_from_scratch.ipynbで紹介したwarm_upの改善版のようなものである。<br>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
import warnings
warnings.simplefilter('ignore')
print("CUDA環境が壊れていないことを祈りながら確認->", torch.cuda.is_available())

CUDA環境が壊れていないことを祈りながら確認-> True


まずは学習につかうTransformerDecoderLayerを定義する。デフォルトにtorch.nn.TransformerDecoderLayerがあるが、<br>
GPTにつかうDecoderLayerは"Attention is all you need"で紹介されているDecoder Layerとは少し異なっている。<br>
そのため、カスタムレイヤーを定義する必要がある。

参考のために比較画像を用意した。

### GPT
<img src = "https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-27_at_12.41.44_PM.png">

### Transformer
<img src = "https://user-images.githubusercontent.com/57289763/160270884-e1901241-a1e6-4890-a5e8-165e87f0c4da.png">

ではGPTのレイヤーをGPTDecoderLayerとして定義しよう。

In [2]:
class GPTDecoderLayer(nn.Module):
    def __init__(self, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0., layer_eps=1e-05, batch_first = False):
        super().__init__()
        self.maskedmultiheadattention = nn.MultiheadAttention(embedding_dim, num_heads,batch_first=batch_first)
        self.dropout_selfattn = nn.Dropout(p = drop_out_rate)
        self.layernorm_selfattn = nn.LayerNorm(embedding_dim, eps = layer_eps)

        self.ffn = nn.Sequential(nn.Linear(embedding_dim, ffn_dim), nn.GELU(), nn.Linear(ffn_dim, embedding_dim))#GELUに変更
        self.layernorm_ffn = nn.LayerNorm(embedding_dim, eps = layer_eps)
        self.dropout_ffn = nn.Dropout(p = drop_out_rate)

    def forward(self, x, pad_mask_self = None, mask_self=None):
        dx, _ = self.maskedmultiheadattention(x,x,x,key_padding_mask = pad_mask_self, attn_mask = mask_self)

        dx = self.dropout_selfattn(dx)

        x = self.layernorm_selfattn(x+dx)

        dx = self.dropout_ffn(self.ffn(x))

        x = self.layernorm_ffn(x + dx)
        return x

製作したGPTDecoderLayerにEmbeddingとPositional EncodingをつければGPTモデルの定義が終わる。<br>
マスクの制作もGPTクラス内部に含める形で実装を行なう。<br>
だがここで注意しなければならなければならないことがある。<br>
TransformerモデルではPositional Encodingはsinとcosを用いて実装したが、<br>
GPTではこのPositional Encodingも学習可能パラメーターとして実装を行なう。<br>
GPTでのこれにあたる層はnn.Embedding(max_sequence_len, embedding_dim)として埋め込み層を定義する。

In [3]:
class GPT(nn.Module):
    def __init__(self, vocab_size, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0.,\
                  layer_eps=1e-05, batch_first = False, T = 10000, N = 1):
        super().__init__()
        #Tはmax_lenを表している
        self.embedding = nn.Embedding(vocab_size, embedding_dim,)
        self.positional_embedding = nn.Embedding(T, embedding_dim)
        self.decoder = nn.ModuleList([GPTDecoderLayer(embedding_dim, ffn_dim, num_heads, drop_out_rate,\
                                                               layer_eps, batch_first) for _ in range(N)])
    def forward(self, x, pad_mask_self = None, mask_self=None):
        x = self.embedding(x)
        pos = torch.arange(0,x.size(1),dtype=torch.long).unsqueeze(0).to(x.device)
        pos = self.positional_embedding(pos)
        x = x + pos
        for layer in self.decoder:
            x = layer(x, pad_mask_self = pad_mask_self, mask_self = mask_self)
        return x
    def create_mask(self, x: torch.tensor, x_pad: int, device: str):
        """
        (batch_size, sequence_length, embedding_dim)の入力を想定
        """
        """
        Trueが無視される値であることに注意すること
        """
        seq_len = x.size(1)
        #srcのマスク制作
        padding_mask = (x == x_pad)
        mask = torch.triu(torch.ones(size = (seq_len, seq_len))==1).transpose(0,1) #下三角行列を作る
        mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask==1.,float(0.0)).to(device)
        return padding_mask, mask
    """To do generate関数を制作する"""

上手く動くか試しに検証してみよう。
入力は(batch_size, tokens) = (1, 6)のLongTensorで、create_maskでmaskを製作し、上手く動作することを確かめる。

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
x = torch.LongTensor([2,10,20,100,512,3]).to(device)
x = x.reshape(1,6)
embedding_size = 768
num_heads = 12
#KarpathyのminGPTを参考に、パラメーターを設定した。
gpt = GPT(50257, embedding_size, embedding_size*4, num_heads, 0.1, batch_first=True, T = 1024, N = 12).to(device)

In [5]:
padding_mask, mask = gpt.create_mask(x, 0, device)
gpt(x)

tensor([[[-0.6247, -1.7400, -0.1319,  ..., -0.7852,  1.2018,  0.1019],
         [-0.3692, -0.6363,  0.5136,  ..., -0.9092,  0.6853, -1.0992],
         [ 0.2017,  0.1197, -1.1565,  ..., -0.7668, -0.5029, -1.0569],
         [ 0.7685, -0.9173, -0.4394,  ..., -0.3718,  0.7840, -1.1355],
         [ 0.1366, -0.9095, -0.0471,  ..., -0.6851,  0.1126,  0.2378],
         [ 0.6988, -0.5835, -0.0877,  ..., -1.7895,  1.5797, -1.8773]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

モデルの構造は以下の通りである。

In [6]:
print(gpt)

GPT(
  (embedding): Embedding(50257, 768)
  (positional_embedding): Embedding(1024, 768)
  (decoder): ModuleList(
    (0-11): 12 x GPTDecoderLayer(
      (maskedmultiheadattention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (dropout_selfattn): Dropout(p=0.1, inplace=False)
      (layernorm_selfattn): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (ffn): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=3072, out_features=768, bias=True)
      )
      (layernorm_ffn): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout_ffn): Dropout(p=0.1, inplace=False)
    )
  )
)


気になるパラメーター数は以下のとおりである。

In [7]:
count_params = 0
for params in gpt.parameters():
    count_params += params.contiguous().view(-1).size(0)
print("The number of parameters is ", count_params)

The number of parameters is  124438272


GPT1のパラメーター数は1億程度であると言われているが、実際に組んだモデルでも1億程度になっていることが確かめられた。<br>
途方もない数字のように思えるが、全てのパラメーターがfloat32(4バイト)であることを考えて<br>簡単な計算を行なうと,
家にある普通のGPUでもモデルのパラメーターを乗せることができるとわかる。<br>

In [8]:
#念の為メモリを開放しておく
import gc
del gpt
del x
del padding_mask, mask
gc.collect()
torch.cuda.empty_cache()

以上でモデルの定義は終わりである。ここからはデータセットの定義に移る。

In [9]:
#本当は実装したかったけどヤムナシ
from transformers import OpenAIGPTTokenizer